In [8]:
import os
import requests as rq
import pandas as pd
import numpy as np
import time
import json
import re
import winsound
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select 
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm,trange
from PIL import Image

In [9]:
Jump = []

In [10]:
root = 'https://www.vscinemas.com.tw/vsweb'  # get img url
path = r"C:\Users\User\Desktop\Hot_Movie_Img" # save in local 所以斜線方向不一樣

Type = {
    'movieTypeId':[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],
    'movieTypeName':['奇幻','冒險','動作','劇情','犯罪','懸疑',
                     '驚悚','歷史','傳記','恐怖','愛情','動畫',
                     '推理','溫馨','家庭','喜劇','暴力','勵志',
                     '紀錄','音樂']
}
Type_CSV = pd.DataFrame(Type)
Type_CSV.to_csv("movieType.csv",index=False,encoding="utf_8_sig")

Grade = {
    'gradeId':[1,2,3,4,5],
    'gradeName':['general','childview','bigchild','teenager','adult']
    # 普0+  護6+  輔12+  輔15+  限18+     
}
Grade_CSV = pd.DataFrame(Grade)
Grade_CSV.to_csv("grade.csv",index=False,encoding="utf_8_sig")

Movie_Info = {
    'movieId':[], # int 255
    'movieName':[], # CH Name string 30
    'movieStart':[],# string 11
    'gradeId':[], # gradeId eg 分級
    'director':[],'actors':[],'movieTypeId':[], # typeId eg 愛情 熱血 動作...
    'movieTime':[],# 片長  
    'movieImg':[], # url_type
    'movieInfo':[],
    'moreMovieImg':[],
    'moreMovieImgNum':[]
}

showing = {
    'cinemaId':[],
    'movieId':[],
    'versionId':[],
    'showingDate':[],
    'startTime':[],
    'theaterId':[]
}

In [11]:
cinemaTable = {} 
with open('cinema.json') as f:               # 引用影城 Id Name
    cinema = json.load(f)   
for i in range(len(cinema['cinemaId'])):
    cinemaTable[cinema['cinemaName'][i]] = cinema['cinemaId'][i]

versionTable = {}
with open('version.json') as f:               # 引用版本 Id Name
    version = json.load(f)
for i in range(len(version['versionId'])):
    versionTable[version['versionName'][i]]= version['versionId'][i]

In [12]:
def Movie(content,i):
    global Movie_Info,Type,Grade,Jump
    try:
        ## movieTime  這个先判斷，如果為空，代表已下架
        Times = content.select('div.movieTime article div.movieDay h4')  # 去對應tag找內容 有另一種寫法是像下面醬
        MoviePlayTime = set()                                            # Times = content.find_all(div,{"class":movieTime})
        for t in Times:
            MoviePlayTime.add(t.text)
        if not MoviePlayTime: # 表示近期沒有撥放 以下架
            Jump.append(i)
            return False
        movieName = content.select_one('div.titleArea h1').text # 電影名稱 中文
        movieName = re.sub(r'[ ．–·，〜!‧:：　-]|『.*',"",movieName) # 正則表達 去掉全部特殊符
        movieName = movieName.upper()
        # 全形轉半形
        tmpName = movieName
        movieName=""
        for s in tmpName:
            ch_num = ord(s)
            if(ch_num>=65281 and ch_num<=65347):
                ch_num -=65248
            movieName += chr(ch_num)
        movieName = movieName.replace('"',"")
        #EN_Name = content.select_one('div.titleArea h2').text
        movieStart = content.select_one('div.titleArea time').text.replace("上映日期：","") # 上映日期

        ## markArea Level
        grade = content.select('div.markArea span')[0].get('class') # eg 保護級 是 list
        if grade != "":
            gradeId = Grade['gradeId'][Grade['gradeName'].index(f'{grade[0]}')]
        else:
            gradeId = 0
        #Mark2 = content.select('div.markArea span')[1].get('class') # eg 熱售

        ## infoArea Director, Actor, Type, L_of_File
        inforArea =content.select_one('div.infoArea table')
        if inforArea != None:  # 直播類型沒有下面這些
            director = content.select_one('div.infoArea table tr td p').text # 導演
            actors = content.select('div.infoArea table tr')[1].select_one('td p').text   # 主演
            movieTime = content.select('div.infoArea table tr')[3].select('td')[1].text # 片長
            typeform = content.select('div.infoArea table tr')[2].select('td')[1].text[0:2] # 我只有選一個 也可以都拿~~
            if typeform != "": # 有的電影沒有類型
                movieTypeId = Type['movieTypeId'][Type['movieTypeName'].index(f'{typeform}')]
            else:
                movieTypeId = 0
        else:
            director = ""
            actors = ""
            movieTime = ""
            typeform = ""
            movieTypeId = 0
        ## Version
    #         VersionList = content.select('div.movieVersion ul li a.versionFirst')
    #         Version = []
    #         for v in VersionList:
    #             Version.append(v.text)   
        ## IMG
        Img = content.select_one('div.movieMain figure img').get('src').replace('..',root)
        Img = rq.get(Img)  # 拿到Img
        if not os.path.exists(path):
            os.mkdir(path)
        with open(f"{path}\_{movieName}_{i}.jpg","wb") as file:
            file.write(Img.content)
            Img = Image.open(f"{path}\_{movieName}_{i}.jpg")
            ReImg = Img.resize((500,740))  # 修正大小
            ReImg.save(f"{path}\_{movieName}_{i}.jpg")
            Img_URL = f"{path}\_{movieName}_{i}.jpg"
        ## movieStory
        movieInfo = content.select_one('div.movieStory p').text

        ## morePicture
        More_Picture = content.select('div.moviePicture a')
        moreImg = ""
        moreImgNum = 0
        if More_Picture:
            for img in More_Picture:
                moreImgNum += 1
                img_url = img.get('href').replace("..",root)
                img = rq.get(img_url)
                if not os.path.exists(path+f'\more_{movieName}_{i}'):  # 建立dir
                    os.mkdir(path+f'\more_{movieName}_{i}')
                with open(f"{path}\more_{movieName}_{i}\_{moreImgNum}.jpg","wb") as file:
                    file.write(img.content)
                    Img = Image.open(f"{path}\more_{movieName}_{i}\_{moreImgNum}.jpg")
                    ReImg = Img.resize((800,300))
                    ReImg.save(f"{path}\more_{movieName}_{i}\_{moreImgNum}.jpg")
                    #moreImg.append(f"{path}\more_{movieName}_{i}\_{moreImgNum}.jpg")
            moreImg = path+f'\more_{movieName}_{i}'
        ########### save into dict 
    #         for ver in Version:
        Movie_Info['movieId'].append(i)
        Movie_Info['movieName'].append(movieName)
        #Movie_Info['EN_Name'].append(EN_Name)
        Movie_Info['movieStart'].append(movieStart)
        Movie_Info['gradeId'].append(gradeId)
        #Movie_Info['Mark2'].append(Mark2)
        Movie_Info['director'].append(director)
        Movie_Info['actors'].append(actors)
        Movie_Info['movieTypeId'].append(movieTypeId)
        Movie_Info['movieTime'].append(movieTime)
        #Movie_Info['Version'].append(ver)
        Movie_Info['movieImg'].append(Img_URL)
        Movie_Info['movieInfo'].append(movieInfo)
        Movie_Info['moreMovieImg'].append(moreImg)
        Movie_Info['moreMovieImgNum'].append(moreImgNum)
        return True
    except:
        print("Error ID:",i)
        return False

In [13]:
# 上面一層 是電影ID
# get each movie in every cinema showing
_waitTime = 0.5
_start = 5937
_end = 6300
driver = webdriver.Chrome(ChromeDriverManager().install()) # install driver
for i in tqdm(range(_start,_end)):
    global Movie_Info,Type,Grade,Jump
    driver.get(f"https://www.vscinemas.com.tw/vsweb/film/detail.aspx?id={i}")
    content = BeautifulSoup(driver.page_source, "html.parser")
    existMovie = Movie(content,i)
    if existMovie:
        time.sleep(_waitTime) # wait 
        listVersion = driver.find_elements(By.CSS_SELECTOR,"section.movieDetail div.movieVersion ul.versionList li a[href='#']")
        #time.sleep(_waitTime) # wait 
        for versionClick in listVersion:
            #time.sleep(_waitTime)
            try:
                versionClick.click()
                time.sleep(_waitTime)
                listCinema = driver.find_elements(By.CSS_SELECTOR,"section.movieDetail div.movieVersion ul.versionList li.show ul li a")
                #time.sleep(_waitTime) # wait 
                for cinemaClick in listCinema:
                    #time.sleep(_waitTime)
                    try:
                        cinemaClick.click()
                        time.sleep(_waitTime)  # wait
                        content = BeautifulSoup(driver.page_source,"html.parser")

                        cinema = content.select_one('div.theaterSelect h1 span').text
                        originalCinema = cinema
                        cinema = re.sub(r'\(.*\)', '', cinema).replace(" ","")
                        cinema = re.sub('[a-zA-Z]','',cinema) # 統一名稱
                        if cinema in cinemaTable.keys():          # 跳過台北松仁
                            cinemaId = cinemaTable[cinema]
                        else:
                            print("Jump",cinema)
                            continue
                        version = content.select_one('div.theaterSelect h1').text.replace(f"{originalCinema}","").replace("/","").replace(" ","")
                        if version in versionTable.keys():
                            versionId = versionTable[version]
                        else:
                            versionId = ""
                        time.sleep(_waitTime)
                        seat = driver.find_elements(By.CSS_SELECTOR,"div.movieTime article.hidden.article.slide div.movieDay ul.bookList a[target='_blank']")
                        #time.sleep(_waitTime) # wait 
                        for seatClick in seat:
                            #time.sleep(_waitTime)
                            try:
                                seatClick.click()
                                time.sleep(_waitTime) # wait
                                driver.switch_to.window(driver.window_handles[-1]) # switch driver to new 
                                time.sleep(_waitTime) # wait
                                content = BeautifulSoup(driver.page_source,"html.parser")
                                DateTime = content.select_one('div.container table tbody tr td div.DivDetail span#LabelSession_dtmDateTime').text[1:]
                                showingDate = DateTime[ :DateTime.find(" ")]
                                startTime = DateTime[DateTime.find(" ")+1:]
                                theaterName = content.select_one('div.container table tbody tr td div.DivDetail span#LabelScreen_strName').text
                                theaterId = re.sub("[第廳館樓]","",theaterName)
                                theaterId = theaterId[theaterId.find(" ")+1:]

                                showing['cinemaId'].append(cinemaId)
                                showing['movieId'].append(i)
                                showing['showingDate'].append(showingDate)
                                showing['startTime'].append(startTime)
                                showing['theaterId'].append(theaterId)
                                showing['versionId'].append(versionId)

                                #print(i,versionId,cinema,version,showingDate,startTime,theaterId)
                                time.sleep(_waitTime)
                                driver.close()
                                time.sleep(_waitTime)
                                driver.switch_to.window(driver.window_handles[-1]) # return driver
                                time.sleep(_waitTime)
                            except:
                                continue
                    except:
                        continue
            except:
                continue
    else:
        continue

C:\Users\User\AppData\Local\Temp\ipykernel_5060\2824417042.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install()) # install driver
  1%|▋                                                                               | 3/363 [00:23<1:02:57, 10.49s/it]

Jump 


 33%|██████████████████████████▏                                                     | 119/363 [21:57<04:05,  1.01s/it]

Jump 


 56%|████████████████████████████████████████████                                  | 205/363 [1:11:42<11:32,  4.38s/it]

...

 80%|███████████████████████████████████████████████████████████▉               | 290/363 [4:47:59<6:31:03, 321.42s/it]

Jump 


 84%|█████████████████████████████████████████████████████████████████▌            | 305/363 [5:00:27<14:30, 15.01s/it]

Jump 


100%|██████████████████████████████████████████████████████████████████████████████| 363/363 [5:07:45<00:00, 50.87s/it]


In [ ]:
frequency = 2000
duration = 5000
winsound.Beep(frequency, duration)

In [14]:
len(showing['cinemaId'])

6462

In [15]:
showing_CSV = pd.DataFrame(showing)
showing_CSV.to_csv("new_showing.csv",index=False,encoding="utf_8_sig")

In [36]:
cinemaTable

{'台北信義威秀影城': 1,
 '台北京站威秀影城': 2,
 '板橋大遠百威秀影城': 4,
 '林口威秀影城': 5,
 '新竹大遠百威秀影城': 7,
 '新竹巨城威秀影城': 9,
 '頭份尚順威秀影城': 10,
 '台中大遠百威秀影城': 11,
 '台中威秀影城': 12,
 '台南大遠百威秀影城': 15,
 '台南南紡威秀影城': 16,
 '高雄大遠百威秀影城': 18,
 '桃園統領威秀影城': 20,
 '花蓮新天堂樂園威秀影城': 21,
 '台南威秀影城': 22,
 '中和環球威秀影城': 25,
 '台中大魯閣新時代威秀影城': 26}

In [41]:
versionTable

{'數位': 1,
 '學優場國': 2,
 'ATMOS': 3,
 'IMAX': 4,
 'MAPPA': 5,
 'IMAX3D': 6,
 'LIVE': 7,
 '學優場': 8,
 'PRE': 9,
 '口述影像': 10,
 'GC數位': 11,
 '4DX3D': 12,
 '4DX': 13,
 '數位國': 14,
 '3D數位': 15,
 'A+': 16}

In [51]:
version = {'數位': 1,
 '學優場國': 2,
 'ATMOS': 3,
 'IMAX': 4,
 'MAPPA': 5,
 'IMAX3D': 6,
 'LIVE': 7,
 '學優場': 8,
 'PRE': 9,
 '口述影像': 10,
 'GC數位': 11,
 '4DX3D': 12,
 '4DX': 13,
 '數位國': 14,
 '3D數位': 15,
 'A+': 16,
'應援場':17        }

In [66]:
version_json = {'versionId':Id,'versionName':Valuse} # for other jupyter load
with open("version.json","wt") as f:
    json.dump(version_json,f)  